In [1]:
#Import dependencies
import os
from dotenv import load_dotenv
import getpass

load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash-preview-04-17",
    temperature = 0.6,
    max_tokens=None,
    timeout=None,
    max_retries=2,
) 

In [3]:
messages = [
    ("system", "You are a helpful assistant that translates English to Shona."),
    ("human", "I love you mom."),
]

response = llm.invoke(messages)
print(response.content)

The most common and respectful way to say "I love you mom" in Shona is:

**Ndinokuda, Amai.**

You could also use:

*   **Ndinokuda, Mhamha.** (Slightly more informal/affectionate term for mom)


In [4]:
#Creating chains in langchain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system" , "You are a helpful assistant that suggests a single fancy restaurant name."),
    ("human" , "I want to open a restaurant for {cuisine} food.")
])

chain_1 = prompt | llm

prompt_2 = ChatPromptTemplate.from_messages([
    ("system" , "You are a helpful assistant that suggests menu items for a given restaurant."),
    ("human" , "Give me items for {restaurant} restaurant as comma seperated list.")
])

chain_2 = prompt_2 | llm

overall_chain = chain_1 | chain_2

response2 = overall_chain.invoke({
    "cuisine" : "Mexican",
})

In [5]:
print(response2.content)

Obsidian Shard Crisps, Smoked Mushroom Caps, Volcanic Ash Steak, Squid Ink Pasta, Ember Roasted Root Vegetables, Dark Chocolate Lava Cake, Black Sesame Ice Cream


In [6]:
#To create a sequential chain 
from langchain_core.runnables import RunnablePassthrough , RunnableParallel
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system" , "You are a helpful assistant that suggests a single fancy restaurant name."),
    ("human" , "I want to open a restaurant for {cuisine} food.")
])

chain_1 = prompt | llm | output_parser

prompt_2 = ChatPromptTemplate.from_messages([
    ("system" , "You are a helpful assistant that suggests menu items for a given restaurant."),
    ("human" , "Give me items for {restaurant} restaurant as comma seperated list.")
])

chain_2 = prompt_2 | llm | output_parser

step_1 = RunnablePassthrough.assign(
    restaurant = chain_1   
)

step_2 = RunnableParallel(
    restaurant_name = (lambda x : x["restaurant"]),
    menu_items = chain_2 ,

)

overall_chain = step_1 | step_2  (lambda x : x["menu_items"]) | output_parser

response2 = overall_chain.invoke({
    "cuisine" : "Mexican",
})



TypeError: 'RunnableParallel' object is not callable

In [ ]:
print(response2)

{'restaurant_name': '**Oro de México**', 'menu_items': 'Guacamole, Nachos, Quesadillas, Sopa de Tortilla, Taco Salad, Carne Asada Tacos, Al Pastor Tacos, Fish Tacos, Enchiladas Rojas, Enchiladas Verdes, Carne Asada Burrito, Carnitas Burrito, Chicken Fajitas, Steak Fajitas, Chile Relleno, Carnitas Plate, Carne Asada Plate, Camarones a la Diabla, Molcajete, Combination Plate, Flan, Churros, Margaritas, Horchata'}
